In [1]:
import requests,pandas as pd
import numpy as np

import json
from pandas import json_normalize

#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process
from dash import Dash, dcc, html
import plotly.express as px

import numpy_financial as npf

import math
import statistics
import numpy as np
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pathlib import Path

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

# Display full number versus scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('JMACData01.xlsx', engine='xlsxwriter')

In [3]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

In [4]:
def clean_percent(x):
    """ If the value is a string, then remove percentage symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('%', '').replace(',', ''))
    return(x)

In [5]:
def clean_negative(x):
    """ If the value is a string, then remove percentage symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('(', '-').replace(')', ''))
    return(x)

In [6]:
#TODO: Webscrape internet article - Used Car Price Trends
UsedCars = pd.read_html("https://www.iseecars.com/articles/best-used-cars-under-15000")

print("25 Best Used Cars Under $15,000")
print()
BestUsedCars = pd.concat(UsedCars)
BestUsedCars = BestUsedCars.reset_index()


25 Best Used Cars Under $15,000



In [7]:
SmallCar15K = pd.DataFrame()
SmallCar15K = BestUsedCars.loc[[1, 2, 3, 4, 5, 6, 7, 8]]
SmallCar15K = SmallCar15K.rename(columns=SmallCar15K.iloc[0]).drop(SmallCar15K.index[0])
SmallCar15K['Average Used Car Price'] = SmallCar15K['Average Used Car Price'].apply(clean_currency).astype('int')
SmallCar15K = SmallCar15K.reset_index(drop=True)
SmallCar15K['Size'] = 'Small Car'
SmallCar15K[['Year', 'Make', 'Model']] = SmallCar15K['Vehicle'].str.split(' ', expand=True)
print("Best Small Cars Under $15,000:")

Best Small Cars Under $15,000:


In [8]:
MidSedan15K = pd.DataFrame()
MidSedan15K = BestUsedCars.loc[[10, 11, 12, 13, 14, 15, 16, 17, 18]]
MidSedan15K = MidSedan15K.rename(columns=MidSedan15K.iloc[0]).drop(MidSedan15K.index[0])
MidSedan15K['Average Used Car Price'] = MidSedan15K['Average Used Car Price'].apply(clean_currency).astype('int')
MidSedan15K = MidSedan15K.reset_index(drop=True)
MidSedan15K['Size'] = 'Midsize Sedan'
MidSedan15K[['Year', 'Make', 'Model']] = MidSedan15K['Vehicle'].str.split(' ', expand=True)
print("Best Midsize Sedans Under $15,000:")

Best Midsize Sedans Under $15,000:


In [9]:
LgSedan15K = pd.DataFrame()
LgSedan15K = BestUsedCars.loc[[20, 21, 22]]
LgSedan15K = LgSedan15K.rename(columns=LgSedan15K.iloc[0]).drop(LgSedan15K.index[0])
LgSedan15K['Average Used Car Price'] = LgSedan15K['Average Used Car Price'].apply(clean_currency).astype('int')
LgSedan15K = LgSedan15K.reset_index(drop=True)
LgSedan15K['Size'] = 'Large Sedan'
LgSedan15K[['Year', 'Make', 'Model']] = LgSedan15K['Vehicle'].str.split(' ', expand=True)
print("Best Large Sedans Under $15,000:")

Best Large Sedans Under $15,000:


In [10]:
SUV15K = pd.DataFrame()
SUV15K = BestUsedCars.loc[[24, 25, 26, 27, 28, 29]]
SUV15K = SUV15K.rename(columns=SUV15K.iloc[0]).drop(SUV15K.index[0])
SUV15K['Average Used Car Price'] = SUV15K['Average Used Car Price'].apply(clean_currency).astype('int')
SUV15K = SUV15K.reset_index(drop=True)
SUV15K['Size'] = 'SUV'
SUV15K[['Year', 'Make', 'Model']] = SUV15K['Vehicle'].str.split(' ', expand=True)
print("Best SUV Under $15,000:")

Best SUV Under $15,000:


In [11]:
Hybrid15K = pd.DataFrame()
Hybrid15K = BestUsedCars.loc[[31, 32, 33, 34]]
Hybrid15K = Hybrid15K.rename(columns=Hybrid15K.iloc[0]).drop(Hybrid15K.index[0])
Hybrid15K['Average Used Car Price'] = Hybrid15K['Average Used Car Price'].apply(clean_currency).astype('int')
Hybrid15K = Hybrid15K.reset_index(drop=True)
Hybrid15K['Size'] = 'Hybrid'
Hybrid15K[['Year', 'Make', 'Model', 'Class']] = Hybrid15K['Vehicle'].str.split(' ', expand=True)
Hybrid15K = Hybrid15K.drop(['Class'], axis=1)
print("Best Hybrid Under $15,000:")

Best Hybrid Under $15,000:


In [12]:
BestCar15K = pd.concat([SmallCar15K, MidSedan15K, LgSedan15K, SUV15K, Hybrid15K], axis=0)
BestCar15K = BestCar15K[['Rank', 'Vehicle', 'iSeeCars Quality Score (of 10)', 'Average Used Car Price', 'Size', 'Year', 'Make', 'Model']]
cols = ['CatRank', 'Vehicle Name', 'ISC Score', 'Avg UCP', 'Category', 'Year', 'Make', 'Model']
BestCar15K.columns = cols
BestCar15K['ISC Score'] = BestCar15K['ISC Score'].astype('float')

BestCar15K = BestCar15K.reset_index(drop=True)

In [13]:
Make_BestCar15K = BestCar15K.groupby('Make').mean()
Cat_BestCar15K = BestCar15K.groupby('Category').mean()
Year_BestCar15K = BestCar15K.groupby('Year').mean()

Factors_BestCar15K = pd.DataFrame()
Factors_BestCar15K = pd.concat([Make_BestCar15K, Cat_BestCar15K, Year_BestCar15K], axis=0)
Factors_BestCar15K['ScoreRank'] = Factors_BestCar15K['ISC Score'].rank(ascending=False)
Factors_BestCar15K['PriceRank'] = Factors_BestCar15K['Avg UCP'].rank(ascending=True)
Factors_BestCar15K['AvgRanking'] = (Factors_BestCar15K['ScoreRank'] + Factors_BestCar15K['PriceRank']) / 2
Factors_BestCar15K['FactorRank'] = Factors_BestCar15K['AvgRanking'].rank(ascending=True)
Factors_BestCar15K = Factors_BestCar15K.sort_values(by=['FactorRank'], ascending=True)
Factors_BestCar15K = Factors_BestCar15K.astype(float)
Factors_BestCar15K.to_excel(writer, sheet_name='Sheet1A-Factors15K')
print("Impact of factors on the 15K Car Selection List")
Factors_BestCar15K

Impact of factors on the 15K Car Selection List


,ISC Score,Avg UCP,ScoreRank,PriceRank,AvgRanking,FactorRank
Toyota,8.775,13367.750,4.000,2.000,3.000,1.000
Mazda,8.700,13377.500,6.000,3.000,4.500,2.000
Small Car,8.886,13881.714,1.000,10.000,5.500,3.000
Hyundai,8.650,13547.500,8.000,6.000,7.000,4.500
Midsize Sedan,8.625,13457.125,9.000,5.000,7.000,4.500
Kia,8.533,13389.667,12.000,4.000,8.000,6.000
Honda,8.875,14289.000,2.000,15.000,8.500,7.500
2014,8.600,13606.375,10.000,7.000,8.500,7.500
2016,8.700,14008.222,6.000,12.000,9.000,9.500
Ford,8.233,13185.333,17.000,1.000,9.000,9.500


In [14]:
BestCar15K['Score%'] = BestCar15K['ISC Score'] / 10
BestCar15K['UCP%'] = 1 - (BestCar15K['Avg UCP'] / 15000)
BestCar15K['FinalRate'] = (BestCar15K['Score%'] + BestCar15K['UCP%']) / 2
BestCar15K = BestCar15K.sort_values(by='FinalRate', ascending=False)
BestCar15K.to_excel(writer, sheet_name='Sheet1B-15KCars')
BestCar15K.head()


,CatRank,Vehicle Name,ISC Score,Avg UCP,Category,Year,Make,Model,Score%,UCP%,FinalRate
11,5,2014 Kia Optima,8.500,12330,Midsize Sedan,2014,Kia,Optima,0.850,0.178,0.514
10,4,2014 Subaru Legacy,8.500,12467,Midsize Sedan,2014,Subaru,Legacy,0.850,0.169,0.509
8,2,2014 Toyota Camry,9.000,13291,Midsize Sedan,2014,Toyota,Camry,0.900,0.114,0.507
6,7,2016 Kia Soul,8.700,12891,Small Car,2016,Kia,Soul,0.870,0.141,0.505
24,3,2014 Toyota Prius,8.400,12529,Hybrid,2014,Toyota,Prius,0.840,0.165,0.502


In [15]:
CarList = pd.DataFrame()
CarList = BestCar15K[['Vehicle Name', 'Year', 'Make', 'Model', 'Avg UCP']]
CarList['SafetyURL'] = 'https://api.nhtsa.gov/SafetyRatings/modelyear/' + CarList['Year'] + '/make/' + CarList['Make'] + '/model/' + CarList['Model']
CarList.to_excel(writer, sheet_name='Sheet2-15KList')
CarList




<ipython-input-15-4b9ac16fc6a2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CarList['SafetyURL'] = 'https://api.nhtsa.gov/SafetyRatings/modelyear/' + CarList['Year'] + '/make/' + CarList['Make'] + '/model/' + CarList['Model']


,Vehicle Name,Year,Make,Model,Avg UCP,SafetyURL
11,2014 Kia Optima,2014,Kia,Optima,12330,https://api.nhtsa.gov/SafetyRatings/modelyear/...
10,2014 Subaru Legacy,2014,Subaru,Legacy,12467,https://api.nhtsa.gov/SafetyRatings/modelyear/...
8,2014 Toyota Camry,2014,Toyota,Camry,13291,https://api.nhtsa.gov/SafetyRatings/modelyear/...
6,2016 Kia Soul,2016,Kia,Soul,12891,https://api.nhtsa.gov/SafetyRatings/modelyear/...
24,2014 Toyota Prius,2014,Toyota,Prius,12529,https://api.nhtsa.gov/SafetyRatings/modelyear/...
12,2014 Mazda Mazda6,2014,Mazda,Mazda6,12769,https://api.nhtsa.gov/SafetyRatings/modelyear/...
0,2014 Honda Civic,2014,Honda,Civic,13524,https://api.nhtsa.gov/SafetyRatings/modelyear/...
16,2014 Ford Taurus,2014,Ford,Taurus,12686,https://api.nhtsa.gov/SafetyRatings/modelyear/...
1,2016 Honda Fit,2016,Honda,Fit,13891,https://api.nhtsa.gov/SafetyRatings/modelyear/...
20,2014 Hyundai Tucson,2014,Hyundai,Tucson,13065,https://api.nhtsa.gov/SafetyRatings/modelyear/...


In [16]:
#TODO: Webscrape The Average Salary by Age in the U.S.

USSalary = pd.read_html("https://smartasset.com/retirement/the-average-salary-by-age#:~:text=As%20you%20might%20imagine%2C%20the,out%20to%20%2429%2C432%20per%20year")
print("The Average Salary by Age Group in the U.S. with 10%-15% monthly salary amount for max car payment")
print()
SalaryGroups = pd.concat(USSalary)
SalaryGroups = SalaryGroups.loc[[1, 2, 3, 4, 5, 6, 7, 8]]
SalaryGroups = SalaryGroups.reset_index()
SalaryGroups = SalaryGroups.rename(columns=SalaryGroups.iloc[0]).drop(SalaryGroups.index[0])
SalaryGroups['Average Salary'] = SalaryGroups['Average Salary'].apply(clean_currency).astype('int')
SalaryGroups = SalaryGroups.reset_index()
SalaryGroups['Average Month'] = SalaryGroups['Average Salary'] / 12
SalaryGroups['10%Salary'] = SalaryGroups['Average Month'] * 0.1
SalaryGroups['15%Salary'] = SalaryGroups['Average Month'] * 0.15
SalaryGroups.to_excel(writer, sheet_name='Sheet3-AgeSalary')
SalaryGroups

The Average Salary by Age Group in the U.S. with 10%-15% monthly salary amount for max car payment



,index,1,Age Group,Average Salary,Average Month,10%Salary,15%Salary
0,1,2,16-19,29432,2452.667,245.267,367.900
1,2,3,20-24,34684,2890.333,289.033,433.550
2,3,4,25-34,49920,4160.000,416.000,624.000
3,4,5,35-44,58604,4883.667,488.367,732.550
4,5,6,45-54,59904,4992.000,499.200,748.800
5,6,7,55-64,59540,4961.667,496.167,744.250
6,7,8,65+,52416,4368.000,436.800,655.200


In [17]:
input1 = pd.read_html("https://www.nerdwallet.com/article/loans/auto-loans/average-car-loan-interest-rates-by-credit-score")
FICO_interest = pd.concat(input1)
FICO_interest_clean = FICO_interest.head()
FICO_final = FICO_interest_clean.dropna(axis=1)

FICO_final['Average APR, new car'] = FICO_final['Average APR, new car'].apply(clean_percent)
FICO_final['Average APR, used car'] = FICO_final['Average APR, used car'].apply(clean_percent)
FICO_final = FICO_final.set_index('Credit score')

FICO_final['Average APR, new car'] = FICO_final['Average APR, new car'].str[:-1]
FICO_final['Average APR, new car'] = FICO_final['Average APR, new car'].astype(float)
FICO_final['Average APR, used car'] = FICO_final['Average APR, used car'].str[:-1]
FICO_final['Average APR, used car'] = FICO_final['Average APR, used car'].astype(float)
FICO_final.to_excel(writer, sheet_name='Sheet4-FICO')
FICO_final

<ipython-input-17-3debb9a3c4a4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FICO_final['Average APR, new car'] = FICO_final['Average APR, new car'].apply(clean_percent)
<ipython-input-17-3debb9a3c4a4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FICO_final['Average APR, used car'] = FICO_final['Average APR, used car'].apply(clean_percent)


,"Average APR, new car","Average APR, used car"
Credit score,,
Superprime: 781-850.,2.470,3.610
Prime: 661-780.,3.510,5.380
Nonprime: 601-660.,6.070,9.800
Subprime: 501-600.,9.410,15.960
Deep subprime: 300-500.,12.530,19.870


In [18]:
#TODO: Webscrape internet article - Facts + Statistics: Teen drivers
TeenStats = pd.read_html("https://www.iii.org/fact-statistic/facts-statistics-teen-drivers")

TeenDrivers = pd.concat(TeenStats)
TeenDrivers = TeenDrivers.reset_index()

YD_Fatalities = pd.DataFrame()
YD_Fatalities = TeenDrivers.loc[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
YD_Fatalities = YD_Fatalities.dropna(axis='columns')
YD_Fatalities = YD_Fatalities.astype(float)
YD_Fatalities.to_excel(writer, sheet_name='Sheet5-Fatalities')
print("Fatalities in Crashes Involving Young Drivers, 2010-2019")
print("")
YD_Fatalities

Fatalities in Crashes Involving Young Drivers, 2010-2019



,index,Nonoccupants,Occupants of other vehicles,Passengers of young drivers,Total,Year,Young drivers age 15–20
0,0.000,493.000,1250.000,1333.000,5041.000,2010.000,1965.000
1,1.000,473.000,1122.000,1194.000,4782.000,2011.000,1993.000
2,2.000,502.000,1230.000,1060.000,4672.000,2012.000,1880.000
3,3.000,469.000,1133.000,1069.000,4367.000,2013.000,1696.000
4,4.000,454.000,1093.000,1015.000,4285.000,2014.000,1723.000
5,5.000,533.000,1326.000,982.000,4744.000,2015.000,1903.000
6,6.000,598.000,1348.000,1033.000,4895.000,2016.000,1916.000
7,7.000,574.000,1396.000,986.000,4800.000,2017.000,1844.000
8,8.000,562.000,1318.000,919.000,4528.000,2018.000,1729.000
9,9.000,506.000,1367.000,880.000,4356.000,2019.000,1603.000


In [32]:
YD_FMVC = pd.DataFrame()
YD_FMVC = TeenDrivers.loc[[21, 22, 23, 24, 25, 26, 27, 28]]
YD_FMVC = YD_FMVC.dropna(axis='columns')

YD_FMVC.to_excel(writer, sheet_name='Sheet6-FMVC')
print("Drivers In Fatal Motor Vehicle Crashes By Age, 2019")
print("")
YD_FMVC

Drivers In Fatal Motor Vehicle Crashes By Age, 2019



,index,"(Drivers in fatal crashes, Involvement rate (1))","(Drivers in fatal crashes, Number)","(Licensed drivers, Number)","(Licensed drivers, Percent of total drivers)","(Unnamed: 0_level_0, Age group)"
21,0,32.500,3892,11992727.000,5.2%,16 to 20
22,1,32.300,4590,14223656.000,6.2,21 to 24
23,2,26.100,10507,40298969.000,17.6,25 to 34
24,3,21.900,8301,37989286.000,16.6,35 to 44
25,4,19.800,7532,38092538.000,16.7,45 to 54
26,5,18.000,7166,39740652.000,17.4,55 to 64
27,6,15.200,4404,29005252.000,12.7,65 to 74
28,7,18.700,3229,17292831.000,7.6,Over 74


In [20]:
#TODO: Webscrape Average Cost of Full Coverage after Adding Teen Driver
TeenRates = pd.read_html("https://www.carinsurance.com/Articles/adding-teen-driver-cost.aspx")

print("Average Cost of Full Coverage after Adding Teen Driver")
print()
StateTeenRates = pd.concat(TeenRates)

StateTeenRates['Avg Premium'] = StateTeenRates['Avg Premium'].apply(clean_currency).astype('int')
StateTeenRates['Avg Premium with Teens'] = StateTeenRates['Avg Premium with Teens'].apply(clean_currency).astype('int')
StateTeenRates['% Increase'] = StateTeenRates['% Increase'].apply(clean_percent).astype('float')
StateTeenRates['Teen Diff'] = StateTeenRates['Avg Premium with Teens'] - StateTeenRates['Avg Premium']

StateTeenRates['PlusTeenRank'] = StateTeenRates['Avg Premium with Teens'].rank()
StateTeenRates['PayExtraRank'] = StateTeenRates['Teen Diff'].rank()
StateTeenRates = StateTeenRates.set_index('State')
StateTeenRates.to_excel(writer, sheet_name='Sheet7-StateRates')
StateTeenRates.head(10)

Average Cost of Full Coverage after Adding Teen Driver



,Avg Premium,Avg Premium with Teens,% Increase,Teen Diff,PlusTeenRank,PayExtraRank
State,,,,,,
Alaska,1236,2532,105.000,1296,14.000,16.000
Alabama,1485,2827,90.000,1342,24.000,20.000
Arkansas,1485,3138,111.000,1653,35.000,39.000
Arizona,1453,3381,133.000,1928,44.000,47.000
California,2001,4238,112.000,2237,50.000,50.000
Colorado,1941,3571,84.000,1630,47.000,36.000
Connecticut,1436,2747,91.000,1311,21.000,17.000
District of Columbia,1679,3236,93.000,1557,37.000,33.000
Delaware,1560,2836,82.000,1276,25.000,15.000


In [27]:
#TODO: Webscrape Used Car Price Trends
UsedPrices = pd.read_html("https://www.cargurus.com/Cars/price-trends/")


CGUsedPrices = pd.concat(UsedPrices)
CGUsedPrices = CGUsedPrices[CGUsedPrices['Unnamed: 0'] .isna()]
CGUsedPrices = CGUsedPrices.drop(['Unnamed: 0'], axis=1)
CGUsedPrices['Avg Price'] = CGUsedPrices['Avg Price'].apply(clean_currency).astype('int')
CGUsedPrices['Last 30 Days'] = CGUsedPrices['Last 30 Days'].apply(clean_percent).astype('float')
CGUsedPrices['L30D Price'] = CGUsedPrices['Avg Price'] * (1 + (CGUsedPrices['Last 30 Days'] / 100))
CGUsedPrices['Last 90 Days'] = CGUsedPrices['Last 90 Days'].apply(clean_percent).astype('float')
CGUsedPrices['L90D Price'] = CGUsedPrices['Avg Price'] * (1 + (CGUsedPrices['Last 90 Days'] / 100))
CGUsedPrices['YoY'] = CGUsedPrices['YoY'].apply(clean_percent).astype('float')
CGUsedPrices['Year1'] = CGUsedPrices['Avg Price'] * 0.81
CGUsedPrices['Year2'] = CGUsedPrices['Avg Price'] * 0.69
CGUsedPrices['Year3'] = CGUsedPrices['Avg Price'] * 0.58
CGUsedPrices['Year4'] = CGUsedPrices['Avg Price'] * 0.49
CGUsedPrices['Year5'] = CGUsedPrices['Avg Price'] * 0.40
CGUsedPrices['Y5Loss'] = CGUsedPrices['Avg Price'] - CGUsedPrices['Year5']

CGUsedPrices = CGUsedPrices.round(2)
CGUsedPrices.to_excel(writer, sheet_name='Sheet8-UsedPrices')
print("CarGurus - Used Car Price Trends & Depreciation")
print()
print("After Depreciation Values: Year 1 (81%); Year 2 (69%); Year 3 (58%); Year 4 (49%); Year 5 (40%)")
print()
CGUsedPrices

CarGurus - Used Car Price Trends & Depreciation

After Depreciation Values: Year 1 (81%); Year 2 (69%); Year 3 (58%); Year 4 (49%); Year 5 (40%)



,Make,Avg Price,Last 30 Days,Last 90 Days,YoY,L30D Price,L90D Price,Year1,Year2,Year3,Year4,Year5,Y5Loss
0,CarGurus Index,30795,0.800,0.150,24.810,31041.360,30841.190,24943.950,21248.550,17861.100,15089.550,12318.000,18477.000
2,Wagon,21245,1.620,5.340,31.970,21589.170,22379.480,17208.450,14659.050,12322.100,10410.050,8498.000,12747.000
3,Pickup Truck,37876,0.270,-0.020,12.570,37978.270,37868.420,30679.560,26134.440,21968.080,18559.240,15150.400,22725.600
4,Convertible,41539,1.600,0.290,17.970,42203.620,41659.460,33646.590,28661.910,24092.620,20354.110,16615.600,24923.400
5,Sedan,23377,0.830,1.680,28.850,23571.030,23769.730,18935.370,16130.130,13558.660,11454.730,9350.800,14026.200
6,Hatchback,17486,0.450,2.680,33.280,17564.690,17954.620,14163.660,12065.340,10141.880,8568.140,6994.400,10491.600
7,Coupe,40226,1.730,1.790,23.160,40921.910,40946.050,32583.060,27755.940,23331.080,19710.740,16090.400,24135.600
8,Van,34171,1.430,7.740,47.050,34659.650,36815.840,27678.510,23577.990,19819.180,16743.790,13668.400,20502.600
9,SUV,39781,0.250,0.900,22.890,39880.450,40139.030,32222.610,27448.890,23072.980,19492.690,15912.400,23868.600
10,Crossover,27598,-0.010,-0.170,24.060,27595.240,27551.080,22354.380,19042.620,16006.840,13523.020,11039.200,16558.800


In [22]:
Vehicle_Type = pd.DataFrame()
Vehicle_Type = CGUsedPrices.loc[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
Vehicle_Type = Vehicle_Type.dropna(axis='columns')
Vehicle_Type = Vehicle_Type.set_index('Make')
Vehicle_Type = Vehicle_Type.astype({'Avg Price':'float','Last 30 Days':'float', 'Last 90 Days':'float','YoY':'float', 'L30D Price':'float','L90D Price':'float', 
                                   'Year1':'float','Year2':'float', 'Year3':'float','Year4':'float', 'Year5':'float', 'Y5Loss':'float'})
Vehicle_Type = Vehicle_Type.round(2)
Vehicle_Type.to_excel(writer, sheet_name='Sheet8A-UP-VehicleType')
Vehicle_Type

,Avg Price,Last 30 Days,Last 90 Days,YoY,L30D Price,L90D Price,Year1,Year2,Year3,Year4,Year5,Y5Loss
Make,,,,,,,,,,,,
Van,34171.000,1.430,7.740,47.050,34659.650,36815.840,27678.510,23577.990,19819.180,16743.790,13668.400,20502.600
Hatchback,17486.000,0.450,2.680,33.280,17564.690,17954.620,14163.660,12065.340,10141.880,8568.140,6994.400,10491.600
Coupe,40226.000,1.730,1.790,23.160,40921.910,40946.050,32583.060,27755.940,23331.080,19710.740,16090.400,24135.600
Convertible,41539.000,1.600,0.290,17.970,42203.620,41659.460,33646.590,28661.910,24092.620,20354.110,16615.600,24923.400
Sedan,23377.000,0.830,1.680,28.850,23571.030,23769.730,18935.370,16130.130,13558.660,11454.730,9350.800,14026.200
Pickup Truck,37876.000,0.270,-0.020,12.570,37978.270,37868.420,30679.560,26134.440,21968.080,18559.240,15150.400,22725.600
Wagon,21245.000,1.620,5.340,31.970,21589.170,22379.480,17208.450,14659.050,12322.100,10410.050,8498.000,12747.000
Crossover,27598.000,-0.010,-0.170,24.060,27595.240,27551.080,22354.380,19042.620,16006.840,13523.020,11039.200,16558.800
Minivan,24212.000,0.080,1.010,32.960,24231.370,24456.540,19611.720,16706.280,14042.960,11863.880,9684.800,14527.200


In [23]:
Vehicle_Make = pd.DataFrame()
Vehicle_Make = CGUsedPrices.loc[[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                                34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
                                55, 56, 57, 58, 59]]
Vehicle_Make = Vehicle_Make.dropna(axis='columns')
Vehicle_Make = Vehicle_Make.set_index('Make')
Vehicle_Make = Vehicle_Make.astype({'Avg Price':'float','Last 30 Days':'float', 'Last 90 Days':'float','YoY':'float', 'L30D Price':'float','L90D Price':'float', 
                                   'Year1':'float','Year2':'float', 'Year3':'float','Year4':'float', 'Year5':'float', 'Y5Loss':'float'})
Vehicle_Make = Vehicle_Make.round(2)
Vehicle_Make.to_excel(writer, sheet_name='Sheet8B-UP-VehicleMake')
Vehicle_Make

,Avg Price,Last 30 Days,Last 90 Days,YoY,L30D Price,L90D Price,Year1,Year2,Year3,Year4,Year5,Y5Loss
Make,,,,,,,,,,,,
Acura,29007.000,0.480,3.160,22.970,29146.230,29923.620,23495.670,20014.830,16824.060,14213.430,11602.800,17404.200
Alfa Romeo,36165.000,0.080,-1.210,16.430,36193.930,35727.400,29293.650,24953.850,20975.700,17720.850,14466.000,21699.000
Aston Martin,137983.000,1.790,2.410,15.970,140452.900,141308.390,111766.230,95208.270,80030.140,67611.670,55193.200,82789.800
Audi,36570.000,-0.020,1.330,22.650,36562.690,37056.380,29621.700,25233.300,21210.600,17919.300,14628.000,21942.000
Bentley,132475.000,1.160,1.310,11.500,134011.710,134210.420,107304.750,91407.750,76835.500,64912.750,52990.000,79485.000
BMW,35501.000,0.370,0.840,22.700,35632.350,35799.210,28755.810,24495.690,20590.580,17395.490,14200.400,21300.600
Buick,22981.000,1.450,4.080,26.620,23314.220,23918.620,18614.610,15856.890,13328.980,11260.690,9192.400,13788.600
Cadillac,37794.000,0.910,2.280,30.590,38137.930,38655.700,30613.140,26077.860,21920.520,18519.060,15117.600,22676.400
Chevrolet,30601.000,0.740,-0.540,25.030,30827.450,30435.750,24786.810,21114.690,17748.580,14994.490,12240.400,18360.600


In [24]:
import webbrowser
print("Display Total Cost of Ownership from Insurance.com")
# generate an URL
#url = 'https://www.insurance.com/total-cost-of-ownership'
#webbrowser.open(url)

ownership_data = pd.read_html("https://www.insurance.com/total-cost-of-ownership")
df_ownership_data = pd.concat(ownership_data)

df_ownership_data = df_ownership_data.replace(np.nan, 0)
df_ownership_data['Total including all costs over five years'] = df_ownership_data['Total including all costs over five years'].apply(clean_currency).astype('int')
df_ownership_data['Sales tax rate'] = df_ownership_data['Sales tax rate'].apply(clean_percent).astype('float')
df_ownership_data['Sales tax on $15K car'] = (df_ownership_data['Sales tax rate']/100) * 15000
df_ownership_data['Sales tax on $15K car'] = df_ownership_data['Sales tax on $15K car'].astype('float')
df_ownership_data['$15K Extra Costs'] = (df_ownership_data['Sales tax on $15K car'].apply(clean_currency).astype(float) + 
                                         df_ownership_data['Average auto insurance rates'].apply(clean_currency).astype(float) + 
                                         df_ownership_data['Annual cost'].apply(clean_currency).astype(float) + 
                                         df_ownership_data['Average Repair Cost'].apply(clean_currency).astype(float) + 
                                          df_ownership_data['Annual extra costs'].apply(clean_currency).astype(float))
df_ownership_data['$15K Extra Costs'] = df_ownership_data['$15K Extra Costs'].astype(float)
df_ownership_data['$15K 5Y Costs'] = df_ownership_data['$15K Extra Costs'] * 5
df_ownership_data['$15K 5Y Costs'] = df_ownership_data['$15K 5Y Costs'].astype(float)
df_ownership_data['Sales tax on $25K car'] = df_ownership_data['Sales tax on $25K car'].apply(clean_currency).astype('float')
df_ownership_data['Average auto insurance rates'] = df_ownership_data['Average auto insurance rates'].apply(clean_currency).astype('int')
df_ownership_data['Miles'] = df_ownership_data['Miles'].apply(clean_currency).astype('int')
df_ownership_data['Annual cost'] = df_ownership_data['Annual cost'].apply(clean_currency).astype('int')
df_ownership_data['Average Repair Cost'] = df_ownership_data['Average Repair Cost'].apply(clean_currency).astype('float')
df_ownership_data['Annual extra costs'] = df_ownership_data['Annual extra costs'].apply(clean_currency).astype('int')
df_ownership_data['Miles over 5Y'] = df_ownership_data['Miles'] * 5
df_ownership_data['Miles over 5Y'] = df_ownership_data['Miles over 5Y'].astype(float)
df_ownership_data = df_ownership_data.drop(['Compared to National Average of $1,457', 'Avg price per gallon',
                                            'Total including all costs over five years', 'Sales tax on $25K car'], axis=1)

print("Breakdown of ownership by State")
state_ownership = df_ownership_data.groupby(by=['State'], dropna=False).sum()
state_ownership = state_ownership.astype(float)
state_ownership = state_ownership.round(2)
state_ownership.to_excel(writer, sheet_name='Sheet9A-Ownership')
state_ownership.head(20)

Display Total Cost of Ownership from Insurance.com
Breakdown of ownership by State


,Sales tax rate,Average auto insurance rates,MPG,Miles,Annual cost,Average Repair Cost,Annual extra costs,Sales tax on $15K car,$15K Extra Costs,$15K 5Y Costs,Miles over 5Y
State,,,,,,,,,,,
Alabama,9.100,1287.000,18.000,16054.000,2270.000,374.090,141.000,1365.000,5437.090,27185.450,80270.000
Alaska,1.760,1183.000,14.000,9915.000,2461.000,362.120,359.000,264.000,4629.120,23145.600,49575.000
Arizona,8.330,1449.000,17.000,14493.000,2254.000,360.850,205.000,1249.500,5518.350,27591.750,72465.000
Arkansas,9.410,1566.000,15.000,14974.000,2957.000,363.100,308.000,1411.500,6605.600,33028.000,74870.000
California,8.540,1846.000,18.000,14435.000,2907.000,385.420,586.000,1281.000,7005.420,35027.100,72175.000
Colorado,7.520,1761.000,17.000,13443.000,2216.000,375.570,287.000,1128.000,5767.570,28837.850,67215.000
Connecticut,6.350,1640.000,18.000,11595.000,1959.000,393.160,294.000,952.500,5238.660,26193.300,57975.000
Delaware,0.000,1828.000,19.000,14802.000,2329.000,373.940,257.000,0.000,4787.940,23939.700,74010.000
Florida,6.800,2219.000,18.000,11836.000,1850.000,366.680,128.000,1020.000,5583.680,27918.400,59180.000


In [25]:
DrivingAgeState = pd.read_html("https://worldpopulationreview.com/state-rankings/driving-age-by-state")
DrivingAgeState = pd.concat(DrivingAgeState)

print("Driving Age by State - What You Need to Know for Your Teen to Safely and Legally Drive")
DrivingAgeState = DrivingAgeState.set_index('State')
DrivingAgeState = DrivingAgeState.astype({"Learner's Permit":'float', 'Restricted License':'float', 'Full License':'float','Car Rentals Age':'float'})
DrivingAgeState = DrivingAgeState.round(0)
DrivingAgeState.to_excel(writer, sheet_name='Sheet9B-DrivingAge')
DrivingAgeState

Driving Age by State - What You Need to Know for Your Teen to Safely and Legally Drive


,Learner's Permit,Restricted License,Full License,Car Rentals Age
State,,,,
Alabama,15.000,16.000,17.000,20.000
Alaska,14.000,16.000,16.000,21.000
Arizona,16.000,16.000,16.000,20.000
Arkansas,14.000,16.000,18.000,20.000
California,16.000,16.000,17.000,20.000
Colorado,15.000,16.000,17.000,20.000
Connecticut,16.000,16.000,18.000,20.000
Delaware,16.000,16.000,17.000,21.000
Florida,15.000,17.000,18.000,20.000


In [26]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()